In [1]:
# LOAD Packages 
#!pip install uproot
#!pip install pandas
#!pip install awkward
#!pip install matplotlib
import uproot
import pandas as pd
import numpy as np
import awkward as ak
import matplotlib
import tqdm
from mpl_toolkits.mplot3d import Axes3D
#matplotlib.rcParams.update(matplotlib.rcParamsDefault)
#matplotlib.rcParams['text.usetex'] = True
import matplotlib.pyplot as plt
import os
import math
import csv
from mpl_toolkits.axes_grid1 import make_axes_locatable
import ROOT
from ROOT import TFile

In [2]:
# Open File
#edep_sim_file = uproot.open("/global/cfs/cdirs/dune/www/data/2x2/simulation/productions/MiniRun5_1E19_RHC/MiniRun5_1E19_RHC.spill/EDEPSIM_SPILLS/0001000/MiniRun5_1E19_RHC.spill.0001023.EDEPSIM_SPILLS.root")
#edep_sim_file["EDepSimEvents/Event"].show()
#edep_sim_file["EDepSimEvents"][0].num_entries
filestring = "/global/cfs/cdirs/dune/www/data/2x2/simulation/productions/MiniRun5_1E19_RHC/MiniRun5_1E19_RHC.spill/EDEPSIM_SPILLS/0001000/MiniRun5_1E19_RHC.spill.0001023.EDEPSIM_SPILLS.root"
inputFile = TFile("/global/cfs/cdirs/dune/www/data/2x2/simulation/productions/MiniRun5_1E19_RHC/MiniRun5_1E19_RHC.spill/EDEPSIM_SPILLS/0001000/MiniRun5_1E19_RHC.spill.0001023.EDEPSIM_SPILLS.root")
inputTree = inputFile.Get("EDepSimEvents")
entries = inputTree.GetEntriesFast()
print("Number of entries in the tree: ", entries)
event_spill_map = inputFile.Get("event_spill_map")
print("Number of entries in the event spill map: ", event_spill_map.GetEntries())
genieTree = inputFile.Get("DetSimPassThru/gRooTracker")
#event_spill_map.GetValue()
if not event_spill_map:
    spillPeriod_s = 0.
else:
    spillPeriod_s = inputFile.Get("spillPeriod_s").GetVal()
    # for setting t_spill
    spillCounter = -1
    lastSpill = None   

filesize = os.path.getsize(filestring)
print("File size is :", filesize)
#genieTree.Scan("EvtNum")
genieTree.Print()

Number of entries in the tree:  5913
Number of entries in the event spill map:  5913
File size is : 601509012
******************************************************************************
*Tree    :gRooTracker: GENIE event tree rootracker format                     *
*Entries :     5913 : Total =        13628699 bytes  File  Size =    4795669 *
*        :          : Tree compression factor =   2.84                       *
******************************************************************************
*Branch  :EvtFlags                                                           *
*Entries :     5913 : BranchElement (see below)                              *
*............................................................................*
*Br    0 :TObject   : BASE                                                   *
*Entries :     5913 : Total  Size=      83635 bytes  File Size  =       9360 *
*Baskets :        4 : Basket Size=      32000 bytes  Compression=   8.88     *
*...................

Warning in <TClass::Init>: no dictionary for class TG4Event is available
Warning in <TClass::Init>: no dictionary for class pair<string,vector<TG4HitSegment> > is available
Warning in <TClass::Init>: no dictionary for class TG4PrimaryVertex is available
Warning in <TClass::Init>: no dictionary for class TG4Trajectory is available
Warning in <TClass::Init>: no dictionary for class TG4PrimaryParticle is available
Warning in <TClass::Init>: no dictionary for class TG4TrajectoryPoint is available
Warning in <TClass::Init>: no dictionary for class TG4HitSegment is available


In [3]:
meson_pdg={211,-211,130,310,311,321,-321,221,331,421,-421,411,-411, 431,-431}

for jentry in range(100):#tqdm(range(entries)):

    print(" ------------------ Processing entry",jentry)
    #print("Got entry",jentry)
    if genieTree:
        gb = genieTree.GetEntry(jentry)
        #print("Got genie entry",jentry)
        genie_idx = 0
    
    part_pdg = []
    for p in range(genieTree.StdHepN):
        #Get only initial and final state particles
        if genieTree.StdHepStatus[p] == 0 or genieTree.StdHepStatus[p] == 1:
            part_pdg.append(genieTree.StdHepPdg[p])

            #Get the incident neutrino four-vector
            if genieTree.StdHepStatus[p] == 0 and np.abs(genieTree.StdHepPdg[p]) in [12, 14, 16]:
                nu_pdg = genieTree.StdHepPdg[p]

    if abs(nu_pdg) != 14: continue
    count_of_pi0s = part_pdg.count(111)
    if count_of_pi0s != 1: continue
    part_pdg_set = set(part_pdg)
    if part_pdg_set.intersection(meson_pdg) != set(): continue

    genie_str = genieTree.EvtCode.GetString().Data()
    if not "CC" in genie_str: continue
    else:
        print("Part PDG: ",part_pdg)
        print("Neutrino PDG: ",nu_pdg)
        print("Genie string: ",genie_str)

    #nb = inputTree.GetEntry(jentry)
        
    #event = inputTree.Event
    #print("Got event",event.EventId)
#
    #globalVertexID = (event.RunId * 1E6) + event.EventId
#
    #print("Global vertex ID:",globalVertexID)
#
    #if not event_spill_map:
    #    spill_it = globalVertexID
    #    t_spill = 0.
    #else:
    #    spill_it_tobj = event_spill_map.GetValue(f"{event.RunId} {event.EventId}")
    #    spill_it = int(spill_it_tobj.GetName())
    #    if spill_it != lastSpill: # New spill?
    #        spillCounter += 1
    #        lastSpill = spill_it
    #    t_spill = spillCounter * spillPeriod_s * 1E6 # convert to us
#
    #print("event",event.EventId,"in spill",spill_it)

 ------------------ Processing entry 0
 ------------------ Processing entry 1
 ------------------ Processing entry 2
 ------------------ Processing entry 3
 ------------------ Processing entry 4
 ------------------ Processing entry 5
 ------------------ Processing entry 6
 ------------------ Processing entry 7
 ------------------ Processing entry 8
 ------------------ Processing entry 9
 ------------------ Processing entry 10
 ------------------ Processing entry 11
 ------------------ Processing entry 12
 ------------------ Processing entry 13
 ------------------ Processing entry 14
 ------------------ Processing entry 15
 ------------------ Processing entry 16
 ------------------ Processing entry 17
 ------------------ Processing entry 18
 ------------------ Processing entry 19
 ------------------ Processing entry 20
 ------------------ Processing entry 21
 ------------------ Processing entry 22
 ------------------ Processing entry 23
 ------------------ Processing entry 24
 ---------